In [25]:
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [115]:
from datetime import date

max_holding = 100
source = "CSV"

#dateTimeFormat = '%Y-%m-%d'
dateTimeFormat = '%d-%b-%Y'

# filter
days = 30 * 24

# frequency
freq = "1D"
symbol = 'BHARTIARTL'
myDate="31_Mar_2019"
filename ="./data/"+symbol+"_"+myDate+".csv"

# Calculate indicators and fields
%run "freedom_indicators_calculation.ipynb"


########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 3
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

In [148]:
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR

# Math
priceSD = STDDEV(price.close)
priceVAR = VAR(price.close)

delBBUP = (price.bbup - price.close)/(price.bbup-price.bbmid)
delBBLOW = (price.close - price.bblow)/(price.bbmid-price.bblow)

price1dGradient = np.arctan(price.close - price.close.shift(1)) * 180 / np.pi
price2dGradient = np.arctan(price.close - price.close.shift(2)) * 180 / np.pi
price3dGradient = np.arctan(price.close - price.close.shift(3)) * 180 / np.pi
price4dGradient = np.arctan(price.close - price.close.shift(4)) * 180 / np.pi
price5dGradient = np.arctan(price.close - price.close.shift(5)) * 180 / np.pi
price10dGradient = np.arctan(price.close - price.close.shift(10)) * 180 / np.pi
price15dGradient = np.arctan(price.close - price.close.shift(15)) * 180 / np.pi

macd1dGradient = price.macd + 3 * (price.macd - price.macd.shift(1))


ADOSC(price.high, price.low, price.close, price.volume)

macdLR = LINEARREG(price.macd)
macdTSF = TSF(price.macd)

priceTSF = TSF(price.close)
priceLR = LINEARREG(price.close)


trend1D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend1D[price1dGradient >= 45] = Trend.UP
trend1D[price1dGradient <= -45] = Trend.DOWN
trend1D[price1dGradient >= 65] = Trend.SUP
trend1D[price1dGradient <= -65] = Trend.SDOWN

trend3D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend3D[price3dGradient >= 45] = Trend.UP
trend3D[price3dGradient <= -45] = Trend.DOWN
trend3D[price3dGradient >= 65] = Trend.SUP
trend3D[price3dGradient <= -65] = Trend.SDOWN

trend5D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend5D[price5dGradient >= 45] = Trend.UP
trend5D[price5dGradient <= -45] = Trend.DOWN
trend5D[price5dGradient >= 65] = Trend.SUP
trend5D[price5dGradient <= -65] = Trend.SDOWN

trend10D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend10D[price10dGradient >= 45] = Trend.UP
trend10D[price10dGradient <= -45] = Trend.DOWN
trend10D[price10dGradient >= 65] = Trend.SUP
trend10D[price10dGradient <= -65] = Trend.SDOWN

trend15D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend15D[price15dGradient >= 45] = Trend.UP
trend15D[price15dGradient <= -45] = Trend.DOWN
trend15D[price15dGradient >= 65] = Trend.SUP
trend15D[price15dGradient <= -65] = Trend.SDOWN

chandlierPoint = price.close-price.atr

sdHighLow = STDDEV((price.high - price.close))
varHighLow = STDDEV((price.high - price.close))
delPriceHighLow = (price.high - price.close)/sdHighLow


# Heikin-Ashi
priceHAOpen = (price.open.shift(1) + price.close.shift(1))/2
priceHAHigh = price.max(axis = 1, skipna = True)
priceHALow = price.min(axis = 1, skipna = True)
priceHAClose = (price.open+price.high+price.low+price.close)/4

In [ ]:
price.dropna(inplace=True)
labels = pd.DataFrame(data = {"holdings":np.zeros(price.index.shape[0])},index=price.index)
#Entry Criterion
labels[
    (delBBUP > 0.15)&(
    ((price.slowk<=10) & (price.close<price.bblow) &(price.RSI<30) ) |
    ((price.slowk<=17) & (price.aroonup>price.aroondown)) |
    (( price.bbup-price.close > 0.1 * (price.bbup - price.bbmid))&(price.macd.shift(3)<0)&(price.macd.shift(2)<0)&(price.macd.shift(1)<0)&(price.macd>=1) &(price.RSI<70)&(price.aroonup>price.aroondown))|
#   ((price.slowk < 15) & ( price.close< price.bbmid - (price.bbmid - price.bblow)*0.2))|
    (((price.aroonup>price.aroondown) | (price.slowk<17)) & (price.close < price.bbmid) & (price['macdhist'] > price['macdhist'].shift(1)) & (price['macdhist'].shift(1) < 0 ) & (price['macdhist'] >= 0 ))
    )
] = S.BUY

#Exit Criterion
labels[
     (((price.slowk.shift(3)>=80) | (price.slowk.shift(2)>=80) | (price.slowk.shift(1)>=80)) & (price.slowk <75) &(price.aroondown>price.aroonup) & (price.RSI>=60) )|
     ((price.bbup - price.close <=-5 ))| 
     ((price.close > price.bbmid + (price.bbup - price.bbmid) * 0.5 ) & (price['macdhist'] < price['macdhist'].shift(1)) & (price['macdhist'].shift(1) > 0 ) & (price['macdhist'] <= 0 ))] = S.SELL


# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

In [34]:
#################################
#   Plotting Data              #
###############################

rangeMin = price.index[-150]
rangeMax = price.index[-1]

%run "freedom_plot.ipynb"

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]



## Entry Criterion
- MACD crossover when market is trending up
- BB when market is moving sideways

## Exit Criterion
- Stochastics

## Identify market trend
- low ATR means market moving sideways
- Flat(gradient=0) MACD means market is moving sideways
- low ATR indicates low volatility - market is moving sideways
- High ATR indicates high volatility - market is trending
- Gradient of current - last 15 days